# Preprocessing tutorial

In [ ]:
from pathlib import Path

import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.widgets as sw

In [ ]:
%matplotlib widget

## Destriping Neuropixels data

Neuropixels probes (and similar) have long shanks that..>


To demonstrate this, we are going to use some test datasets shared by Olivier Winter from IBL.

For more information, please refer to: ...

In [ ]:
base_folder = Path("../../SpikeInterface Dataset Tutorial")

In [ ]:
data_folder = base_folder / "datatset_destriping"
# session = "4c04120d-523a-4795-ba8f-49dbb8d9f63a"
session = "68f06c5f-8566-4a4f-a4b1-ab8398724913"
# session = "8413c5c6-b42b-4ec6-b751-881a54413628"
# session = "f74a6b9a-b8a5-4c80-9c30-7dd4cdbb48c0"


In [ ]:
ibl_data_cbin_file = [p for p in (data_folder / session).iterdir() if p.name.endswith(".ap.cbin")][0]

In [ ]:
recording = se.read_cbin_ibl(cbin_file=ibl_data_cbin_file)
recording

Let's first take a look at the "raw" traces:

In [ ]:
w = sw.plot_traces(recording, scale=2)

We notice there are both some horizontal and vertical "stripes", plus some "patches".

Although Neuropixels data have a hardware highpass filter (cutoff at 150Hz), there can be come "leftover" content from the LFP frequency band.

So let's first highpass the data and see how it looks:

In [ ]:
recording_hp = spre.highpass_filter(recording)
w = sw.plot_traces(recording_hp, scale=5)

Now the "patches" are mainly gone, but we can still see some vertical artefacts and some horizontal lines.

While the vertical lines can be attributed to transient external noise, the horizontal ones are mainly due to a channel either being too silent ("dead") or too "noisy".

In addition, on the top of the probe we can observe an area of faint activity. This could indicate that the channels are not inserted in the brain, which is commont when targeting a surface area.

In [ ]:
# detect bad channels + CMR / highpass_spatial_filter()

In [ ]:
# whitening / zscore?

# mention motion

## Remove artifacts (NP2.0)

In [ ]:
# notch filter
# blank with zeros (best)
# blank with noise

# Save preprocessing

dump_to_json(relative_to=)

# reload with base_folder